In [ ]:
import cv2
import numpy as np
from tensorflow.keras.datasets import cifar100

# Load CIFAR-100 dataset
(x_train, y_train), (x_test, y_test) = cifar100.load_data()


In [ ]:
# Function to extract color histogram features
def color_histogram(image):
    # Convert image to HSV color space
    hsv_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    num_bins=256
    # Calculate histograms for each color channel
    hist_h = cv2.calcHist([hsv_image], [0], None, [num_bins], [0, 180])  # Hue
    hist_s = cv2.calcHist([hsv_image], [1], None, [num_bins], [0, 256])  # Saturation
    hist_v = cv2.calcHist([hsv_image], [2], None, [num_bins], [0, 256])  # Value

    # Normalize histograms
    cv2.normalize(hist_h, hist_h, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX)
    cv2.normalize(hist_s, hist_s, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX)
    cv2.normalize(hist_v, hist_v, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX)

    # Concatenate histograms into a single feature vector
    histogram = np.concatenate((hist_h, hist_s, hist_v), axis=0)

    return histogram.ravel()

# Extract features from training and test data
train_features_color = []
for image in x_train:
    feature_c = color_histogram(image)
    train_features_color.append(feature_c)

test_features_color = []
for image in x_test:
    feature_c_test = color_histogram(image)
    test_features_color.append(feature_c_test)

# Convert feature lists to NumPy arrays
train_features_color = np.array(train_features_color)
test_features_color = np.array(test_features_color)

print("Training features shape:", train_features_color.shape)
print("Test features shape:", test_features_color.shape)

Training features shape: (50000, 768)
Test features shape: (10000, 768)


In [ ]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score

In [ ]:
print("Normalizing features...")
scaler = StandardScaler()
color_hist_train = scaler.fit_transform(train_features_color)
color_hist_test = scaler.transform(test_features_color)

print("Training SVM classifier...")
svm = SVC(kernel="linear", C=1.0)
svm.fit(color_hist_train, y_train.ravel())

print("Evaluating on test set...")
test_predictions = svm.predict(color_hist_test)
print("Test Accuracy:", accuracy_score(y_test, test_predictions))

confusion_matrix = confusion_matrix(y_test, test_predictions)
print("confusion matrix:",confusion_matrix)


Normalizing features...
Training SVM classifier...
Evaluating on test set...
Test Accuracy: 0.0852
confusion matrix: [[38  1  2 ...  0  1  0]
 [ 2 12  3 ...  0  0  3]
 [ 4  1 12 ...  0  2  0]
 ...
 [ 1  0  0 ...  1  1  1]
 [ 1  1  5 ...  0 11  1]
 [ 2  3  0 ...  0  3 11]]


In [ ]:
# Function to compute LBP histogram
def lbp_histogram(image):
    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply LBP operator
    lbp = cv2.calcHist([gray], [0], None, [256], [0, 256])
    lbp = cv2.normalize(lbp, lbp, 0, 255, cv2.NORM_MINMAX)

    # Flatten the histogram
    hist = lbp.flatten()

    return hist

# Extract features from training
train_features_lbp = []
for image in x_train:
    feature_lbp = lbp_histogram(image)
    train_features_lbp.append(feature_lbp)

# Extract features from test data
test_features_lbp = []
for image in x_test:
    feature_lbp = lbp_histogram(image)
    test_features_lbp.append(feature_lbp)

# Convert feature lists to NumPy arrays
train_features_lbp = np.array(train_features_lbp)
test_features_lbp = np.array(test_features_lbp)

print("Training features shape:", train_features_lbp.shape)
print("Test features shape:", test_features_lbp.shape)

Training features shape: (50000, 256)
Test features shape: (10000, 256)


In [7]:
print("Normalizing features...")
scaler = StandardScaler()
lbp_hist_train = scaler.fit_transform(train_features_lbp)
lbp_hist_test = scaler.transform(test_features_lbp)

print("Training SVM classifier...")
svm2 = SVC(kernel="linear", C=1.0)
svm2.fit(lbp_hist_train, y_train.ravel())

print("Evaluating on test set...")
test_predictions = svm.predict(lbp_hist_test)

print("Test Accuracy:", accuracy_score(y_test, test_predictions))

confusion_matrix = confusion_matrix(y_test, test_predictions)
print("confusion matrix:",confusion_matrix)

Normalizing features...
Training SVM classifier...
Evaluating on test set...


ValueError: X has 256 features, but SVC is expecting 768 features as input.

In [ ]:
clf = SVC(kernel='linear')  # Linear kernel
# Train the model
clf.fit(train_features_lbp, y_train)
# Make predictions on the test set
y_pred = clf.predict(test_features_lbp)
# Evaluate the model's accuracy
accuracy = accuracy_score(y_test, y_pred)
print("the accuracy of SVM feature with LBP ", accuracy)
confusion_matrix = confusion_matrix(y_test, test_features_lbp)
print(confusion_matrix)

Fitting KMeans on training data...
Predicting clusters for test data...
